## Лабораторная работа 1 модуль
### Ксения Анисимова ЦМГН 
#### Вариант 14

### Класс для габаритов и координат (в см)

In [618]:
from typing import Union

class Coords:
    def __init__(self, x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):  
        return "(x-%s; y-%s; z-%s)" % (self.x, self.y, self.z)  #подумать над разделением

In [619]:
d = Coords(1, 2, 3)
print(d)

(x-1; y-2; z-3)


### Класс кухонного объекта: 
**Координаты**: середина дна гарнитура  
**Габариты**: x - длина, y - высота, z - ширина  
**Методы проверки пересечения**

In [892]:
class Kitchen_Object:
    def __init__(self, size: Coords, coords: Coords):
        self.__size = size
        self.coords = coords
        
    @property
    def size(self):
        return self.__size
    
    @size.setter 
    def size(self, size):
        self.middle_coords()
        size.__ = size 
    
    #метод нахождения координат двух углов проекции (дна) объекта (на одной из диагонали)
    @classmethod
    def corner_coords(self, size, coords):
        x = [coords.x - size.x/2, coords.z + size.z/2]
        z = [coords.x + size.x/2, coords.z - size.z/2]
        return x, z
    
    #координаты середины сторон гарнитура (без высоты) - для проверки, приставлен ли объект к стене 
    def middle_coords(self):
        size = self.__size
        coords = self.coords
        a = [coords.x - size.x/2, coords.z]
        b = [coords.x, coords.z + size.z/2]
        c = [coords.x + size.x/2, coords.z]
        d = [coords.x, coords.z - size.z/2]
        return [a, b, c, d]
    
    #проверка пересечения проекций 
    @classmethod
    def _bottom_overlap(cls, size1, size2, coords1, coords2):
        coords1 = Kitchen_Object.corner_coords(size1, coords1) # координаты по обеим концам диагонали
        top1, bottom1 = coords1[0], coords1[1] # координаты "верхней" и "нижней" точки проекции гарнитура(x;z)
        coords2 = Kitchen_Object.corner_coords(size2, coords2)
        top2, bottom2 = coords2[0], coords2[1]
        if top1[0] == bottom1[0] or top1[1] == bottom1[1] or top2[0] == bottom2[0] or top2[1] == bottom2[1]: #площадь 0
            return False
        if top1[0] > bottom2[0] or top2[0] >  bottom1[0]: #слева/справа
            return False
        if bottom1[1] > top2[1] or bottom2[1] > top1[1]: #сверху/снизу 
            return False
        return True
    
    #проверка пересечения по высоте
    def _height_overlap(self, size1, size2, coords1, coords2):
        height1 = size1.y #
        height2 = size2.y
        y1 = coords1.y
        y2 = coords2.y
        if y1 + height1 < y2:
            return False
        if y2 + height2 < y1:
            return False 
        return True
    
    @classmethod
    def overlap(self, object1, object2):
        if self._bottom_overlap(object1.size, object2.size, object1.coords, object2.coords) == False:
            return False
        if self._height_overlap(self, object1.size, object2.size, object1.coords, object2.coords):
            return True

### Класс шкафа: 
**Координаты и габариты** - наследованы от Kitchen_Object \
Материал сделан приватным из житейской логики

In [893]:
from typing import Union

class Cabinet(Kitchen_Object):
    def __init__(self, material, size, coords):
        super().__init__(size, coords)
        self.material = material

    def __repr__(self):  
        return "Cabinet: material - %s, size%s, coordinates%s)" % (self.material, self.size, self.coords)

    @property
    def material(self):
        return self.__material
    
    @material.setter 
    def material(self, material: Union[str, int]):
        self.__material = material

In [894]:
cab1 = Cabinet("wood", Coords(50,100,50), Coords(25,0,25))
cab2 = Cabinet("oak", Coords(30,50,30), Coords(25,10,100))
cab3 = Cabinet("birch", Coords(30,50,30), Coords(25,10,50))
print(cab1, cab2, cab3, sep="\n")
print(Kitchen_Object.overlap(cab1, cab2))
print(Kitchen_Object.overlap(cab1, cab3))

Cabinet: material - wood, size(x-50; y-100; z-50), coordinates(x-25; y-0; z-25))
Cabinet: material - oak, size(x-30; y-50; z-30), coordinates(x-25; y-10; z-100))
Cabinet: material - birch, size(x-30; y-50; z-30), coordinates(x-25; y-10; z-50))
False
True


### Класс техники: 
**Координаты и габариты** - наследованы от Kitchen_Object  
Имя сделано приватным из житейской логики  
Необходимость и смысл метода включения и выключения техники неясны, поэтому он не реализован

In [895]:
class Appliance(Kitchen_Object):
    def __init__(self, name, size, coords):
        super().__init__(size, coords)
        self.__name = name

    @property
    def name(self):
        return self.__name
    
    @name.setter 
    def name(self, name: Union[str, int]):
        self.__name = name

    def __repr__(self):  
        return  "Appliance: name - %s, size%s, coordinates%s" % (self.name, self.size, self.coords)
    
    #не поняла, зачем тут метод включения/выключения техники
    def switch_on_off(self):
        pass

In [896]:
app1 = Appliance("TV", Coords(20,20,40), Coords(0,50,60))
app2 = Appliance("Oven", Coords(30,10,30), Coords(30,50,60))
print(app1)
print(app2)
print(Appliance.overlap(cab1, cab2))

Appliance: name - TV, size(x-20; y-20; z-40), coordinates(x-0; y-50; z-60)
Appliance: name - Oven, size(x-30; y-10; z-30), coordinates(x-30; y-50; z-60)
False


### Класс кухни: 
**Габариты (х, y)** - высоту не учитываем  
**Содержание** - список шкафов и техники

In [897]:
from collections import namedtuple
from functools import reduce
import math

kitchen_size = namedtuple('KitchenSize', ['x', 'y'])

class Kitchen:
    def __init__(self, size: kitchen_size, content: list):
        self.__size = size
        self.content = content 
        
    @property
    def size(self):
        return self.__size
    
    @size.setter 
    def size(self, size: kitchen_size):
        self.__size = size
    
    def __repr__(self):  
        return "Kitchen set: \n %s, \n content: %s \n" % (self.__size, self.content)

        
#см. метод fit_check()       
#     @property
#     def content(self):
#         return self.__content

#     @content.setter 
#     def content(self, *kitchen_object: Union[Cabinet, Appliance]):
#         content = []
#         flag = True 
#         middle_coords = reduce(lambda x, y: x+y, kitchen_Object.middle_coords()) #делаем из многоуровневого списка одноуровневый
#         for i in range(0, len(middle_coords), 2): #нечетные (по х)
#             if middle_coords[i] < 0 or middle_coords[i] > self.__size.x:
#                 print("This object doesn't fit the kitchen")
#                 flag = False
#         for i in range(1, len(middle_coords), 2): #нечетные (по z)
#             if middle_coords[i] < 0 or middle_coords[i] > self.__size.z:
#                 print("This object doesn't fit the kitchen")
#                 flag = False 
#         if flag:
#             self.__content = content.append(kitchen_object)
     
    #def __find_appliance(self, content: list):
    def find_appliance(self, content: list):
        appliances = []
        for x in content:
            if isinstance(x, Appliance):
                appliances.append(x)
        return appliances
    
    #def __find_cabinet(self, content):
    def find_cabinet(self, content):
        cabinets = []
        for x in content:
            if isinstance(x, Cabinet):
                cabinets.append(x)
        return cabinets
    
    def fit_check(self):
        kitchen_set = self.content
        removed = set()
        for el in kitchen_set:
            middle_coords = reduce(lambda x, y: x+y, el.middle_coords())
            x_coords = middle_coords[::2]
            z_coords = middle_coords[1::2]
            for i in x_coords:
                if i < 0 or i > self.__size.x:
                    removed.add(el)
            for j in z_coords:
                if j < 0 or j > self.__size.y:
                    removed.add(el)
        for el in removed:
            if el in kitchen_set:
                kitchen_set.remove(el)
        print("\033[1m"+ "These objects go beyond the kitchen boundaries and were removed from kitchen set:" + "\033[0m", *removed, sep="\n")
        self.content = kitchen_set
        
# Метод для проверки, не висит ли техника в воздухе:
#     - если координата по высоте = 0, то не висит
#     - если больше нуля, то проверяем, не стоит ли техника на шкафе
#     - будем считать, что техника не может висеть на стене

    def appliance_float(self):
        res = []
        appliance = self.find_appliance(self.content)
        cabinets = self.find_cabinet(self.content)
        for app in appliance:
            if app.coords.y == 0:
                res.append("%s doesn't float" % app.name)
            else:
                counter = 0
                for cab in cabinets:
                    if Kitchen_Object._bottom_overlap(cab.size, app.size, cab.coords, app.coords) and \
                    app.coords.y == cab.size.y + cab.coords.y or app.coords.x == 0 or app.coords.z == 0:
                        counter = counter + 1
                if counter >= 1:
                    res.append("%s doesn't float" % app.name)
                else:
                    res.append("%s floats!" % app.name)
        return res 
    
    def same_material_cabinets_too_close(self):
        res = set()
        cabinets = self.find_cabinet(self.content)
        for i in range(0, len(cabinets), 1):
            for j in range(0, len(cabinets), 1):
                if cabinets[i].material == cabinets[j].material and i != j \
                and math.dist([cabinets[i].coords.x, cabinets[i].coords.z], [cabinets[j].coords.x, cabinets[j].coords.z]) \
                < 200:
                    res.add(cabinets[i].material)
        for x in res:
            print("Cabinets of %s are too close!" % x)
            
    def overlap(self):
        counter = 0 
        res = set()
        for x in self.content:
            for y in self.content:
                if x != y and Kitchen_Object.overlap(x, y) == True:
                    if (y, x) not in res:
                        res.add((x, y))
        return res

In [898]:
def print_set(res):
    for x in res:
        print(*x, sep="\n")

### Создаем тестовую мебель и технику для кухни

In [899]:
cabinet1 = Cabinet("oak", Coords(10, 30, 20), Coords(5, 10, 30))
cabinet2 = Cabinet("oak", Coords(20, 30, 10), Coords(20, 0, 5))
appl1 = Appliance("TV", Coords(10, 10, 10), Coords(25, 30, 5))
appl2 = Appliance("Oven", Coords(10, 10, 10), Coords(45, 30, 25))
appl3 = Appliance("Stove", Coords(10, 10, 10), Coords(30, 0, 45))
appl4 = Appliance("Grill", Coords(-10, -10, -10), Coords(-30, 0, -45))
appl5 = Appliance("Hood", Coords(-10, -10, -10), Coords(1000, 1000, 1000))

print(cabinet1.middle_coords())

[[0.0, 30], [5, 40.0], [10.0, 30], [5, 20.0]]


### Создаем тестовую кухню и проверяем по условиям задачи

In [900]:
content = [cabinet1, cabinet2, appl1, appl2, appl3, appl4, appl5]
size = kitchen_size(200, 200)
k = Kitchen(size, content)
print(k)

Kitchen set: 
 KitchenSize(x=200, y=200), 
 content: [Cabinet: material - oak, size(x-10; y-30; z-20), coordinates(x-5; y-10; z-30)), Cabinet: material - oak, size(x-20; y-30; z-10), coordinates(x-20; y-0; z-5)), Appliance: name - TV, size(x-10; y-10; z-10), coordinates(x-25; y-30; z-5), Appliance: name - Oven, size(x-10; y-10; z-10), coordinates(x-45; y-30; z-25), Appliance: name - Stove, size(x-10; y-10; z-10), coordinates(x-30; y-0; z-45), Appliance: name - Grill, size(x--10; y--10; z--10), coordinates(x--30; y-0; z--45), Appliance: name - Hood, size(x--10; y--10; z--10), coordinates(x-1000; y-1000; z-1000)] 



In [901]:
k.fit_check()

print("\033[1m" + "\nAppliances position:" + '\033[0m')
print(*k.appliance_float(), sep="\n")

print("\033[1m" + "\nCabinets of the same material that are less than 2m close:" + '\033[0m')
k.same_material_cabinets_too_close()

print("\033[1m" + "\nOverlap occured among these pieces of furniture:" + '\033[0m')
print_set(k.overlap())

These objects go beyond the kitchen boundaries and were removed from kitchen set:
Appliance: name - Hood, size(x--10; y--10; z--10), coordinates(x-1000; y-1000; z-1000)
Appliance: name - Grill, size(x--10; y--10; z--10), coordinates(x--30; y-0; z--45)

Appliances position:
TV doesn't float
Oven floats!
Stove doesn't float

Cabinets of the same material that are less than 2m close:
Cabinets of oak are too close!

Overlap occured among these pieces of furniture:
Cabinet: material - oak, size(x-20; y-30; z-10), coordinates(x-20; y-0; z-5))
Appliance: name - TV, size(x-10; y-10; z-10), coordinates(x-25; y-30; z-5)
